<a href="https://colab.research.google.com/github/bjcabral/Coursera_Capstone/blob/master/5312019Coursera_Battle_of_the_Neighborhoods_(Week_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import data and create datframes:
We will be working with two datasets, one from Foursquare and one from the West Virginia Department of Transportation.

#### Dataset description
The Foursquare dataset corresponds to all businesses in a 10-mile radious of the Falling Water's WV town. From this dataset we are looking to learn the number and type of businesses in the area, their popularity, and customer's population, as well as the business proximity to potential locations. From the dataset we will also learn if the identified business are competitors or business that could attract potential fast food customers.

The second dataset contains traffic survey for the areas, specifically, traffic volume by the hours for highway exits near potential locations. We hope to learn the traffic patterns and volume to consider potential customers volume other than local residents.

The following section pulls the datasets and create the corresponding dataframes

In [0]:
# Install necessary libraries 
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install -U folium
print('Folium installed')
#!conda install -c conda-forge geopy --yes
!pip install -U geopy
print('geopy library install')


     |████████████████████████████████| 92kB 5.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.9.1 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


Folium installed
     |████████████████████████████████| 102kB 4.5MB/s 
  Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


geopy library install


In [0]:
# Import necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # Map plotting library
print('Libraries imported')

Libraries imported


In [0]:
#Set up Foursquare credentials
CLIENT_ID = '3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH' # your Foursquare ID
CLIENT_SECRET = 'FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH
CLIENT_SECRET:FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP


Set up variables to be used in the Foursquare search request URL

In [0]:
address = '78 Worchester Dr, Falling Waters, WV 25419' # Nearest address to proposed location, center of plaza
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Proposed location by corporate is  {}, and the longitude is {}".format(latitude, longitude))

Proposed location by corporate is  39.5765995, and the longitude is -77.8773458


The following section sets up the Foursquare URL to search for all business locations in a 10-miles radius. Something to notice here is that we are not searching by search_queary which queries the name of the business, but by category. For this, I had to include the variable &category to the URL and populate the category value to the category for "Fast Food" as found in the Foursquare website.

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&category=`**CATEGORY**`&radius=`**RADIUS**`&limit=`**LIMIT**


In [0]:
search_query = ''
category = '4bf58dd8d48988d16e941735'
radius = 20000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&category={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, category, radius, LIMIT)
url

results = requests.get(url).json()
results

 .... OK!


{'meta': {'code': 200, 'requestId': '5cf16f98db04f52f66d47029'},
 'response': {'confident': True,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '519000ca498e6e48c8844244',
    'location': {'cc': 'US',
     'country': 'United States',
     'distance': 67,
     'formattedAddress': ['WV 25419', 'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.57662842268352,
       'lng': -77.87655725041604}],
     'lat': 39.57662842268352,
     'lng': -77.87655725041604,
     'postalCode': '25419',
     'state': 'West Virginia'},
    'name': "King's Pizza",
    'referralId': 'v-1559326616'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_

In [0]:
# Transforming the data from json to create dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,519000ca498e6e48c8844244,NaN,US,NaN,United States,NaN,67,"[WV 25419, United States]","[{'label': 'display', 'lat': 39.57662842268352...",39.576628,-77.876557,25419,West Virginia,King's Pizza,v-1559326616,NaN
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4e9833694690053a31a53b69,78 Worchester Dr,US,Falling Waters,United States,NaN,103,"[78 Worchester Dr, Falling Waters, WV 25419, U...","[{'label': 'display', 'lat': 39.57708883485135...",39.577089,-77.876325,25419,WV,Food Lion Grocery Store,v-1559326616,NaN
2,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,524c8950498ee4ca9d92dfaa,8360 Williamsport Pike,US,Falling Waters,United States,I-81,56,"[8360 Williamsport Pike (I-81), Falling Waters...","[{'label': 'display', 'lat': 39.57678202423249...",39.576782,-77.877964,25419,WV,AC&T/Subway,v-1559326616,NaN
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4f554bdde4b0a3a6d5d213bf,44 Worcester Ave.,US,Falling Waters,United States,NaN,83,"[44 Worcester Ave., Falling Waters, WV 25419, ...","[{'label': 'display', 'lat': 39.57685148219772...",39.576851,-77.876431,25419,WV,Red Lantern,v-1559326616,NaN
4,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4bc64452bf29c9b6f18df92a,Williamsport Pike,US,Falling Waters,United States,NaN,2347,"[Williamsport Pike, Falling Waters, WV, United...","[{'label': 'display', 'lat': 39.558496, 'lng':...",39.558496,-77.891377,NaN,WV,Little Falls Presbyterian Church,v-1559326616,NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,King's Pizza,Italian Restaurant,NaN,US,NaN,United States,NaN,67,"[WV 25419, United States]","[{'label': 'display', 'lat': 39.57662842268352...",39.576628,-77.876557,25419,West Virginia,519000ca498e6e48c8844244
1,Food Lion Grocery Store,Supermarket,78 Worchester Dr,US,Falling Waters,United States,NaN,103,"[78 Worchester Dr, Falling Waters, WV 25419, U...","[{'label': 'display', 'lat': 39.57708883485135...",39.577089,-77.876325,25419,WV,4e9833694690053a31a53b69
2,AC&T/Subway,Sandwich Place,8360 Williamsport Pike,US,Falling Waters,United States,I-81,56,"[8360 Williamsport Pike (I-81), Falling Waters...","[{'label': 'display', 'lat': 39.57678202423249...",39.576782,-77.877964,25419,WV,524c8950498ee4ca9d92dfaa
3,Red Lantern,Chinese Restaurant,44 Worcester Ave.,US,Falling Waters,United States,NaN,83,"[44 Worcester Ave., Falling Waters, WV 25419, ...","[{'label': 'display', 'lat': 39.57685148219772...",39.576851,-77.876431,25419,WV,4f554bdde4b0a3a6d5d213bf
4,Little Falls Presbyterian Church,Church,Williamsport Pike,US,Falling Waters,United States,NaN,2347,"[Williamsport Pike, Falling Waters, WV, United...","[{'label': 'display', 'lat': 39.558496, 'lng':...",39.558496,-77.891377,NaN,WV,4bc64452bf29c9b6f18df92a
5,falling waters wv,City,NaN,US,NaN,United States,NaN,1513,"[WV 25419, United States]","[{'label': 'display', 'lat': 39.56337243094846...",39.563372,-77.881402,25419,West Virginia,4f258096e4b0dc27ba872952
6,PepsiCo Dream Machine,Miscellaneous Shop,8812 Williamsport Pike,US,Falling Waters,United States,NaN,1287,"[8812 Williamsport Pike, Falling Waters, WV 25...","[{'label': 'display', 'lat': 39.586473, 'lng':...",39.586473,-77.885152,25419,WV,4e52ec09bd415b8727b88dd0
7,Food Kitty Rest Stop,Rest Area,NaN,US,Falling Waters,United States,NaN,120,"[Falling Waters, WV 25419, United States]","[{'label': 'display', 'lat': 39.577233, 'lng':...",39.577233,-77.876201,25419,WV,508ed9bee4b0aaac8af4bd30
8,Convenient Motor Service,Automotive Shop,9 Community Lane,US,Falling Waters,United States,NaN,1627,"[9 Community Lane, Falling Waters, WV 25419, U...","[{'label': 'display', 'lat': 39.58624005317688...",39.586240,-77.863090,25419,WV,4e4a7174aeb7de71b3963dfe
9,Dollar General,Discount Store,NaN,US,Falling Waters,United States,NaN,163,"[Falling Waters, WV, United States]","[{'label': 'display', 'lat': 39.57583288836229...",39.575833,-77.878974,NaN,WV,51a4f71b498e183125c5973f


In [0]:
Falling_Waters_10Miles = dataframe_filtered[['name','categories', 'distance','lat','lng']] # select relevant variables from the dataframe_filtered. Use venue name, category, distance from town, latitude and longitude.
Falling_Waters_10Miles

,name,categories,distance,lat,lng
0,King's Pizza,Italian Restaurant,67,39.576628,-77.876557
1,Food Lion Grocery Store,Supermarket,103,39.577089,-77.876325
2,AC&T/Subway,Sandwich Place,56,39.576782,-77.877964
3,Red Lantern,Chinese Restaurant,83,39.576851,-77.876431
4,Little Falls Presbyterian Church,Church,2347,39.558496,-77.891377
5,falling waters wv,City,1513,39.563372,-77.881402
6,PepsiCo Dream Machine,Miscellaneous Shop,1287,39.586473,-77.885152
7,Food Kitty Rest Stop,Rest Area,120,39.577233,-77.876201
8,Convenient Motor Service,Automotive Shop,1627,39.586240,-77.863090
9,Dollar General,Discount Store,163,39.575833,-77.878974


In [0]:
# Map all the business locations in the 10-miles radius centered around Falling Waters, WV.
FallingWaters_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Falling Waters

# add a red circle marker to represents proposed location
folium.Circle(
    location=[39.5765995, -77.8773458],
    radius=100,
    color='red',
    popup='Falling Waters, WV',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6,
    parseHTML=False).add_to(FallingWaters_map)

# add the proposed location as a blue circle
for lat, lng, label, distance in zip(Falling_Waters_10Miles['lat'], Falling_Waters_10Miles['lng'], Falling_Waters_10Miles['categories'], Falling_Waters_10Miles['distance']):
    miles = distance/1610
    folium.Marker(
        location=[lat, lng],
        radius=5,
        color='blue',
        popup=[label, miles],
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        parseHTML=False).add_to(FallingWaters_map)

# display map
FallingWaters_map